# OBS Demo: A demonstration of standard-cell based obfuscation for imaging security in SKY130

Ckristian Duran, 2024

SPDX-License-Identifier: Apache-2.0

#### Team Members 

|Name|Affiliation|IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|
| Ckristian Duran (Maker) <br /> Email IDs: <br /> ckdur.iso [at] gmail [dot] com <br /> ckristian.duran [at] correo [dot] uis [dot] edu [dot] com <br /> duran [at] silicon [dot] u-tokyo [dot] ac [dot] jp|Universidad Industrial de Santander (UIS) <br /> The University of Tokyo (東京大学)| Yes |Yes |

Replace `[at]` with `@`

Replace `[dot]` with `.`

## Abstract

This project demonstrates a standard-cell based obfuscation of an example digital circuit to protect against imaging tools.
The obfuscator works by replacing cells from a netlist by cells with identical behavior but different layout.
This obfuscation is performed in the post-synthesis digital circuit, and the rest of the steps are executed in a typical digital composition (ej: PnR, DRC, LVS, Simulation, etc).
The alternative cells layout do not have a significant impact in area, but the timing and capacitances can vary a little, making Static Timing Analysis vary between the original circuit and an obfuscated one.
The cell library was previously built and characterized with enough files to perform digital synthesis and implementation.
As part of the demo, an additional imaging-reversal procedure is built by template matching, to try to recognize back the cells in the digital circuit.
This demo is built for Skywater 130nm (SKY130).

## 1. Introduction

Circuit security is a rising concern in recent years. The analysis of digital circuits to extract information has been a common ocurrence in reverse-engineering. Such analysis includes the extraction of a netlist from different stages of design and fabrication. Despite numerous intents to obscure and secretize the process in such stages, attacks always arise at the end of the production cycle. An attacker can perform decaping of a chip and perform image-reversal of a circuit in critical locations to extract information. This project is yet another way to prevent attacks from chip imaging by obfuscating the digital circuit.

### 1.1 Standard cell library

The first component of this project is an standard cell library which contains the obfuscation cells. These cells contains the same circuit but contains different layouts. To exemplify this, lets take a regular NAND cell.

```spice
* This file can be found in obs_demo/lib/sky130.lvs
.SUBCKT ND2D1 a1 a2 vdd vss zn
*.PININFO a1:I a2:I zn:O vdd:B vss:B 
M_M13 zn a1 xi1_n6 vss nfet_01v8 l=1.500e-07 w=6.500e-07
M_M14 xi1_n6 a2 vss vss nfet_01v8 l=1.500e-07 w=6.500e-07
M_M12 zn a2 vdd vdd pfet_01v8 l=1.500e-07 w=1.000e-06
M_M11 zn a1 vdd vdd pfet_01v8 l=1.500e-07 w=1.000e-06
.ENDS
```

This circuit contains 4 transistors, 2 NMOS and 2 PMOS. At the moment of creating the layout, there are different ways to arrange the transistors. In the case of the NAND there are 4 possible arrangements for a minimum space layout:

```
Arrange 1             Arrange 2          Arrange 3          Arrange 4
       a1   a2            a1    a2         a1    a2             a1   a2
P: vdd | z  | vdd     z   | vdd | z      z | vdd | z        vdd | z  | vdd
N: gnd | n6 | z       gnd | n6  | z      z | n6  | gnd      z   | n6 | gnd
       a1   a2            a1    a2         a2    a1             a2   a1
```

With these arrangements, is possible to place and route such cells in 4 different manners in the case of the NAND

![ND2D1](assets/img/ND2D1.toon.png "NAND A.1")![ND2D1_2](assets/img/ND2D1_2.toon.png "NAND A.2")![ND2D1_3](assets/img/ND2D1_3.toon.png "NAND A.3")![ND2D1_1](assets/img/ND2D1_1.toon.png "NAND A.4")

This behaviour can be extended to other cells. In the included library, the cells are limited to 4 different layouts, but this process can be repeated as many times as different arrangements are possible for a cell. This is an example of a D-Flip-Flop.

![DFQD1](assets/img/DFQD1.toon.png "DFQD1 A.1")![DFQD1_1](assets/img/DFQD1_1.toon.png "DFQD1 A.2")

![DFQD1_2](assets/img/DFQD1_2.toon.png "DFQD1 A.3")![DFQD1_3](assets/img/DFQD1_3.toon.png "DFQD1 A.4")

With these cells,  a library is composed which contains combinational and sequential circuitry. Such layouts are going to be randomly swapped in a netlist to obfuscate the digital circuit at the moment of producing a layout.

### 1.2 Obfuscation procedure

The obfuscation consists of perform synthesis, obfuscation, place, and route the circuit with the standard cell library. The next figure details the overall flow of this implementation.

![Obfuscation procedure](assets/img/obs_demo_flow.png "OBS flow")

After getting a netlist of the circuit from synthesis, the obfuscation is performed to the output netlist from [yosys](https://github.com/YosysHQ/yosys).
The script contains a link list to replace the existing cells to any of the 4 possible alternatives (if they exist).
The cell is chosen randomly using the `obs_demo/syn/tcl/aleatorize.py` script.
For example, an instance of `ND2D1` can be replaced by `ND2D1`, `ND2D1_1`, `ND2D1_2`, or `ND2D1_3`.
The output of this script is a obfuscated netlist, where all the cells are replaced with equivalent circuits with theoretical different layouts.

The next steps are included in the typical digital flow to output a macro layout.
Placement, Clock Tree Synthesis (CTS) and route of the circuit is done using the obfuscated library and the geometry abstracts.
Such process are executed with [OpenROAD](https://github.com/The-OpenROAD-Project/OpenROAD) with scripts included in the project.
The output is a definition file which contains placement and routing of the geometry abstract, which is used by [Magic](https://github.com/RTimothyEdwards/magic) to create a final layout GDS file. DRC and LVS can be also performed at this stage.

TODO: The DRC and LVS do not work at this point, as the standard cell library does not pass DRC and has connectivity issues.

### 1.3 Reverse enginering the circuit

After creating the GDS layout of the digital macro, the user can perform a test for a imaging reverse-engineering of the circuit.
Imaging tools are used to create a photo-like realistic image of the layouts of the digital macro, as well as every cell implemented in the obfuscated library.
Afterwards, all the images are imported into a imaging reversing tool named [degate](https://github.com/DegateCommunity/Degate).
This tool allows the user to perform imaging-reversing to a series of die photos to identify digital circuits and their netlists.
The user can perform a template matching of the images of this demo to simulate an attack and evaluate the identification rate at a first iteration.

![Reverse test procedure](assets/img/obs_demo_rev.png "REV flow")

## 2. Contents of the notebook

This notebook contans an step-to-step guide to execute the [obs_demo](https://github.com/ckdur/obs_demo) project.
In the following sections, we will describe to to the following:

1. Install tools and dependencies, working for (almost) every linux platform.
2. Create the obfuscated circuit, which guide the user to perform synthesis, aleatorize, place/CTS/route, and signoff of the digital macro.
3. Perform reverse-engineering using imaging tools, where we guide the user to perform an attack to the image of the layout, either automatically or via scripts.

The demo by itself contains the following:

1. Synthesis scripts.
2. Aleatorize script in the synthesis container.
3. Place and route scripts.
4. Signoff scripts, including GDS creation, DRC, and LVS.
5. Imaging scripts to create photo-like imaging.
6. Reverse-engineering scripts, to execute degate with the previous images.

## 3. Preparing the environment

### 3.1 Installing tools and dependencies 

The following code will allow you to install the dependencies from a new conda environment. Several steps can be skipped if you either installed these requirements locally in your system or in a separate conda environment you already have prepared for digital integration. The checks for these tools are automatically implemented, and this block of code can be ran just as it is.

NOTE: These scripts were executed in a jupyter notebook running locally, and having a vanilla Ubuntu 22.4 LTS.

In [ ]:
import os
import pathlib
import sys

# We are going to use micromamba, as seems to be the most neutral to work in any environment
micromamba_path=str(pathlib.Path('micromamba').resolve())
micromamba=micromamba_path + "/bin/micromamba"
if not os.path.isfile(micromamba):
    !mkdir -p {micromamba_path}/bin
    !cd {micromamba_path} && curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba

# Section for checking the status of the conda installation
conda_prefix_path = os.environ.get('CONDA_PREFIX', None)
conda_exists = True
if conda_prefix_path is None:
    conda_prefix_path = str(pathlib.Path('conda-env').resolve())
    if not os.path.isdir(conda_prefix_path):
        conda_exists = False
%env CONDA_PREFIX={conda_prefix_path}
!mkdir -p {conda_prefix_path}/conda-meta

# Only create the environment if the conda does not exist
if not conda_exists:
    print("Create the conda environment")
    !{micromamba} create --yes --prefix {conda_prefix_path}

# Forcing this conda environment to work with python 3.7
!echo 'python == 3.7*' >> {conda_prefix_path}/conda-meta/pinned

# Set some environment variables
ORIG_PATH=os.environ['PATH']
ORIG_LD_LIBRARY_PATH=os.environ.get('LD_LIBRARY_PATH', '')
PATH = "{}/bin:{}".format(conda_prefix_path, ORIG_PATH)  # Is supposed to not fail
LD_LIBRARY_PATH = "{0}/lib:{0}/lib/python3.7:{1}".format(conda_prefix_path, ORIG_LD_LIBRARY_PATH)
%env PATH={PATH}
%env LD_LIBRARY_PATH={LD_LIBRARY_PATH}

# Now, check progressively for all the dependencies, which will list in this dict
req = {
    "magic": {"cmd": "magic --version", "channel": "main"},
    "klayout": {"cmd": "klayout -v", "channel": "main"},
    "yosys==0.36_8_g45dd9eca6": {"cmd": "yosys --version", "channel": "main"},
    "yosys-symbiflow-plugins==1.20230425_62_g0ad1af2": {"cmd": "[ -f \"$(yosys-config --datdir)/plugins/sdc.so\" ] && true || false ", "channel": "main"},
    "netgen": {"cmd": "which netgen", "channel": "main"},
    "openroad": {"cmd": "openroad -version", "channel": "main"},
    "git\\>=2.34.1": {"cmd": "git --version", "channel": "conda-forge"}
}
installs = list()
channels = set()
for key, elem in req.items():
    ret = os.system(elem["cmd"])
    if ret != 0:
        # This command does not exist, or does not execute. Regardless, install
        channels.add(elem["channel"])
        installs.append(key)
    else:
        # Detected. Just assume is there
        print("Detected!: {}".format(key))

install_str = " ".join(installs)
channel_str = " ".join(["--channel " + c for c in channels])
channel_print = ", ".join(channels)
print("Attempting to install from {}: {}".format(channel_print, install_str))
!{micromamba} install --yes --prefix {conda_prefix_path} --channel litex-hub {channel_str} {install_str}

# Check exclusively for the PDK_ROOT separatelly... I wish there is a better way to check for the PDK
PDK_ROOT = os.environ.get('PDK_ROOT', None)
if PDK_ROOT is None:
    !{micromamba} install --yes --prefix {conda_prefix_path} --channel litex-hub --channel main open_pdks.sky130a
    PDK_ROOT = "{0}/share/pdk".format(conda_prefix_path)
    %env PDK_ROOT={PDK_ROOT}

### 3.2 Download the repository

The code below will download the repository where this demo is hosted.

In [ ]:
import os
import pathlib
import sys

obs_demo_path = str(pathlib.Path('obs_demo').resolve())
obs_demo_settings = obs_demo_path + "/settings.mk"

if not os.path.isdir(obs_demo_path):
    # Download the repository
    !git clone -b main --recursive https://github.com/ckdur/obs_demo.git {obs_demo_path}

%env ROOT_DIR={obs_demo_path}

### 3.3 Build degate and gimp

Gimp and degate are tools that are not listed in conda packages. 

The next block will use the repository to build a separate environment for Degate, and build it.

In [ ]:
# TODO: This unfortunatelly is not possible to run in the previous conda environment. 
# REASON: qt5 and qt6 conflicts.
# WORKAROUND: We need to install another one

# Section for checking the status of the degate conda installation
degate_conda_prefix_path = os.environ.get('DEGATE_CONDA_PREFIX', None)
degate_conda_exists = True
if degate_conda_prefix_path is None:
    degate_conda_prefix_path = str(pathlib.Path('degate-conda-env').resolve())
if not os.path.isdir(conda_prefix_path):
    degate_conda_exists = False
%env DEGATE_CONDA_PREFIX={degate_conda_prefix_path}
!mkdir -p {degate_conda_prefix_path}/conda-meta

# Only create the environment if the conda does not exist
if not degate_conda_exists:
    print("Create the conda environment")
    !{micromamba} create --yes --prefix {degate_conda_prefix_path}

# Forcing this conda environment to work with python 3.7
!echo 'python == 3.8*' >> {degate_conda_prefix_path}/conda-meta/pinned

# Set some environment variables
DEGATE_PATH = "{}/bin:{}".format(degate_conda_prefix_path, ORIG_PATH)  # Is supposed to not fail
DEGATE_LD_LIBRARY_PATH = "{0}/lib:{0}/lib/x86_64-linux-gnu:{0}/lib/python3.8:{1}".format(degate_conda_prefix_path, ORIG_LD_LIBRARY_PATH)
%env PATH={DEGATE_PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}

req = {
    "pkg-config\\>=0.29.2": {"cmd": "pkg-config --version", "channel": "conda-forge"}, 
    "autoconf": {"cmd": "autoconf --version", "channel": "conda-forge"}, 
    "automake": {"cmd": "automake --version", "channel": "conda-forge"},
    "intltool": {"cmd": "intltoolize --version", "channel": "conda-forge"}, 
    "cmake\\>=3.22.1": {"cmd": "cmake --version", "channel": "conda-forge"},
    "gcc\\>=12.3.0": {"cmd": "gcc --version", "channel": "conda-forge"},
    "gxx\\>=12.3.0": {"cmd": "g++ --version", "channel": "conda-forge"},
    "qt6-main\\>=6.6.2": {"cmd": "which qtdiag6", "channel": "conda-forge"},
    "glib": {"cmd": "false", "channel": "conda-forge"},             # FORCED
    "glib-networking": {"cmd": "false", "channel": "conda-forge"},          # FORCED
    "glib-tools": {"cmd": "false", "channel": "conda-forge"},               # FORCED
    "gtk2": {"cmd": "false", "channel": "conda-forge"},                     # FORCED
    "cairo": {"cmd": "false", "channel": "conda-forge"},                    # FORCED
    "lcms2": {"cmd": "false", "channel": "conda-forge"},                    # FORCED
    "libpng": {"cmd": "false", "channel": "conda-forge"},                   # FORCED
    "zlib": {"cmd": "false", "channel": "conda-forge"},                     # FORCED
    "libzlib": {"cmd": "false", "channel": "conda-forge"},                  # FORCED
    "bzip2": {"cmd": "false", "channel": "conda-forge"},                    # FORCED
    "xz": {"cmd": "false", "channel": "conda-forge"},                       # FORCED
    "json5": {"cmd": "false", "channel": "conda-forge"},                    # FORCED
    "json-c": {"cmd": "false", "channel": "conda-forge"},                   # FORCED
    "exiv2": {"cmd": "false", "channel": "conda-forge"},                    # FORCED
    "packaging": {"cmd": "false", "channel": "conda-forge"},                # FORCED
    "iconv": {"cmd": "false", "channel": "conda-forge"},                    # FORCED
    "librsvg": {"cmd": "false", "channel": "conda-forge"},                  # FORCED
    "poppler": {"cmd": "false", "channel": "conda-forge"},                  # FORCED
    "boost": {"cmd": "false", "channel": "conda-forge"},              # FORCED
    "meson": {"cmd": "meson --version", "channel": "conda-forge"},
    "ninja": {"cmd": "ninja --version", "channel": "conda-forge"},
    "imagemagick": {"cmd": "convert --version", "channel": "conda-forge"},
}
installs = list()
channels = set()
for key, elem in req.items():
    ret = os.system(elem["cmd"])
    if ret != 0:
        # This command does not exist, or does not execute. Regardless, install
        channels.add(elem["channel"])
        installs.append(key)
    else:
        # Detected. Just assume is there
        print("Detected!: {}".format(key))

install_str = " ".join(installs)
channel_str = " ".join(["--channel " + c for c in channels])
channel_print = ", ".join(channels)
print("Attempting to install from {}: {}".format(channel_print, install_str))
!{micromamba} install --yes --prefix {degate_conda_prefix_path} --channel litex-hub {channel_str} {install_str}

!mkdir -p $ROOT_DIR/rev/src/Degate/build
!cd $ROOT_DIR/rev/src/Degate/build && cmake $ROOT_DIR/rev/src/Degate -DBoost_USE_STATIC_LIBS=OFF
!make -C $ROOT_DIR/rev/src/Degate/build all

In the same fashion, we conveniently put an script to build GIMP. Would take some time if not installed.

In [ ]:
# Script to compile gimp... Why is this more difficult than running openroad?
# GIMP is very bloated, but, is either this or photoshop
%env PATH={DEGATE_PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}
pkg_config_path=os.environ.get("PKG_CONFIG_PATH", "")
%env PKG_CONFIG_PATH={degate_conda_prefix_path}/share/pkgconfig:{degate_conda_prefix_path}/lib/pkgconfig:{degate_conda_prefix_path}/lib/x86_64-linux-gnu/pkgconfig:/usr/lib/x86_64-linux-gnu/pkgconfig:/usr/lib/pkgconfig:/usr/share/pkgconfig:{pkg_config_path}
ret = os.system("gimp --version")

def build_meson(target, extra=""):
    !rm -rf {target}/_build
    !mkdir -p {target}/_build
    !cd {target}/_build && meson setup .. -Dprefix={degate_conda_prefix_path} {extra}
    !cd {target}/_build && ninja
    !cd {target}/_build && ninja install

def build_automake(target, extra=""):
    !cd {target} && {target}/autogen.sh
    !cd {target} && {target}/configure --prefix={degate_conda_prefix_path} {extra}
    !make -C {target}
    !make -C {target} install

if ret != 0:
    # Ohh no, you do not have gimp. And is not a conda repo... hopefully this works!
    # Otherwise, just run "apt install gimp" or equivalent
    gimp_build = str(pathlib.Path('gimp-build').resolve())
    !mkdir -p {gimp_build}
    !which pkg-config
    !cd {gimp_build} && wget https://download.gimp.org/gimp/v2.10/gimp-2.10.30.tar.bz2
    !cd {gimp_build} && tar -xf {gimp_build}/gimp-2.10.30.tar.bz2
    !git clone https://gitlab.gnome.org/GNOME/babl.git {gimp_build}/babl
    !git clone https://gitlab.gnome.org/GNOME/gegl.git {gimp_build}/gegl
    !git clone https://github.com/mypaint/libmypaint -b v1.5.0 {gimp_build}/libmypaint
    !git clone https://github.com/mypaint/mypaint-brushes -b v1.3.x {gimp_build}/mypaint-brushes
    !cd {gimp_build} && wget https://download.gnome.org/sources/gexiv2/0.14/gexiv2-0.14.2.tar.xz
    !cd {gimp_build} && tar -xf gexiv2-0.14.2.tar.xz
    !git clone https://github.com/GNOME/gobject-introspection.git -b 1.72.0 {gimp_build}/gobject-introspection
    build_meson(gimp_build + "/gobject-introspection")
    build_meson(gimp_build + "/babl")
    !cp {degate_conda_prefix_path}/lib/x86_64-linux-gnu/pkgconfig/babl-0.1.pc {degate_conda_prefix_path}/lib/x86_64-linux-gnu/pkgconfig/babl.pc
    build_meson(gimp_build + "/gegl")
    build_meson(gimp_build + "/gexiv2-0.14.2", "-Dvapi=false -Dpython3=false")
    build_automake(gimp_build + "/mypaint-brushes")
    build_automake(gimp_build + "/libmypaint")
    build_automake(gimp_build + "/gimp-2.10.30", "--disable-python")

## 4. Creating the obfuscated circuit

### 4.1 Synthesis and obfuscation

The synthesis uses `yosys` together with the included standard cell library to create a first netlist which the user will obfuscate later.
The circuit under test will be an [aes core](https://github.com/secworks/aes.git).
Configurations can be found in the `obs_demo/settings.mk`. The user is free to modify this file according to the following specifications:

```Makefile
# Obfuscation levels. This number can be maximum 4
LEVELS?=4
# Recognition levels. Later used by degate to use just the plain library (1) 
# or all the cells (4). Can be maximum $(LEVELS)
RECLEVELS?=1

# Proportions of the layout. The macro will be a 4:2 ratio with 80% density in this case.
PX?=4
PY?=2
PR?=0.8

# When performing the imaging reversal, these are the dimmensions to analyze
DISPX=100
DISPY=100
DISPW=100
DISPH=100

# TOP name of the digital circuit
TOP?=aes

# A list of verilog files for the digital circuit
SYN_SRC?=$(ROOT_DIR)/aes/src/rtl/aes.v
```

The following block will perform the synthesis using `yosys`. After synthesizing, the script will automatically run the `aleatorize.py` script to scramble the cells with the alternatives.

In [ ]:
%env PATH={PATH}
%env LD_LIBRARY_PATH={LD_LIBRARY_PATH}

# Perform Synthesis
!make -C $ROOT_DIR/syn all

After the user execute this command, the differences between the `obs_demo/syn/outputs/aes_net.v` and `obs_demo/syn/outputs/aes_net_obs.v` can be seen:

![Demo of the synthesis](assets/img/post_synthesis.png "Post synthesis result")

### 4.2 Place and route, and signoff

The place and route procedure will take the `obs_demo/syn/outputs/aes_net_obs.v` netlist and the standard cell library in the `obs_demo/lib`. To execute this, run the following:

In [ ]:
%env PATH={PATH}
%env LD_LIBRARY_PATH={LD_LIBRARY_PATH}

# Perform PnR
!make -C $ROOT_DIR/pnr all

After this process finishes, the user can start the signoff process.
To create the gds, and perform DRC/LVS, please run the following cell, but note please that DRC and LVS are not passing at the moment.

In [ ]:
%env PATH={PATH}
%env LD_LIBRARY_PATH={LD_LIBRARY_PATH}

# Create the GDS
!make -C $ROOT_DIR/signoff gds

# Perform DRC (disabled)
#!make -C $ROOT_DIR/signoff drc

# Perform LVS (disabled)
#!make -C $ROOT_DIR/signoff lvs

## 5. Reverse-engineering the circuit

To demonstrate the obfuscation impact in the circuit, the next steps are going to demonstrate an hypotetical scenario where the attacker extracts images from a chip and perform an imaging attack. This process is accomplished by creating photo-like images of each one of the standard cells, and also an image of the overall circuit.

### 5.1 Imaging the circuits

This step involves the conversion from the layouts of both standard cells and the top module. The next cell will take the GDS file of each of those circuits and extract images. The procedure is as following:

1. Use klayout with an special layer file to color the layout with a different pallete that resembles more the post-fabrication.
2. Add different kinds of image manipulation using GIMP to add some texture and 3D effects.
3. Convert the output of all the post-effect GIMP images into different TIF images which can be fed into degate.

In [ ]:
%env PATH={DEGATE_PATH}:{PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}:{LD_LIBRARY_PATH}

# Make the TOP photo-like
!make -C $ROOT_DIR/images top_real

The output should be something like this for the TOP:

![Image TOP](assets/img/aes.r.full.png "Image TOP")

In the case of the standard cells, we extract some additional features from the ports of the standard cells. This process is necessary to detect interconnections in the future to extract a netlist.

In [ ]:
%env PATH={DEGATE_PATH}:{PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}:{LD_LIBRARY_PATH}

# Make the standard cells photo-like, and also create the library definition for degate
!make -C $ROOT_DIR/images lib_real
!make -C $ROOT_DIR/images lib_out

The standard cells are now extracted in images ready to setup the attack.

![Image ND2D1](assets/img/ND2D1_logic.png "Photo")![Image ND2D1_1](assets/img/ND2D1_1_logic.png "Photo")![Image ND2D1_2](assets/img/ND2D1_2_logic.png "Photo")![Image ND2D1_3](assets/img/ND2D1_3_logic.png "Photo")

### 5.2 Setup the attack

The conditions now are set to start an attack into the simulated chip photo. At this stage, an attacker would use the degate program manually to create a project where all these photos are introduced one by one.

In this case, the project provides a procedure to create the project automatically with the insertion of the cells.

In [ ]:
%env PATH={DEGATE_PATH}:{PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}:{LD_LIBRARY_PATH}

# Create the degate project
!make -C $ROOT_DIR/images lib_install

If the user were to run manually the degate utility and open `obs_demo/rev/aes`, would see an screen with the previous imaging implemented, and also the standard cell library loaded and ready to be matched. If the user runs `make -C obs_demo/rev manual`, it can create an instance of Degate that looks like this:

![Screen Degate](assets/img/screen_degate.png "Degate")

At this point, the attacker would run `Matching -> Template Matching`, selecting the cells and performing the attack. The next block does this process automatically:

In [ ]:
%env PATH={DEGATE_PATH}:{PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}:{LD_LIBRARY_PATH}

# Start the matching process
!make -C $ROOT_DIR/rev run_template

### 5.3 Judging the success of the attack

Finally, this project can evaluate the success of the attack. The following block extracts the matching information from degate and compares it with the actual instances inside of the signoff GDS. It also creates a new GDS highlighting the matching and reports a final mathing result.

In [ ]:
%env PATH={DEGATE_PATH}:{PATH}
%env LD_LIBRARY_PATH={DEGATE_LD_LIBRARY_PATH}:{LD_LIBRARY_PATH}

# Evaluate the template
!make -C $ROOT_DIR/images eval

The end of this script will create a file in `obs_demo/images/real_4/aes.stat` that looks like the following:

```
Statistics
  detected = 31/181 (17.1%)
  wrong = 98/181 (54.1%)
  not detected = 152/181 (84.0%)
```

Additional to the report, the user can run separatelly `make -C obs_demo/images view_extract` which would display a klayout window, with red/green/cyan boxes. In this context, the meaning of the colors are:

- RED: Not recognized
- GREEN: Succesfully recognized
- CYAN: Wrongly recognized cells

![Klayout view](assets/img/klayout_view.png "Recognition in Klayout")

## 6. Conclusions and acknoledgements

This project implemented a digital circuit example (an aes core) with an obfuscation that involves an special standard cell library.
This standard cell library contains different layouts of the same circuit.
With this approach, the project is able to mitigate image-reversing of the circuit.
The concept is partially proven by creating photo-like images of the layouts and the standard cells.
Such images are fed into a image-reversing tool named `degate` which performs correlation detection in the different photos.
If we suppose the attacker only has knoledge of the basic standard cell library, the attack is only successful around the 20% of the total circuitry.

The project was possible thanks to the usage of an standard cell generator developed in the "Universidad Industrial de Santander" (Insdustrial University of Santander) in Colombia/Bucaramanga.
I also want to acknoledge my current job at the University of Tokyo.

## 7. References and suggestions

- This notebook was inspired by `VLSI23/accepted_notebooks/3LFCC`. This projects uses the dependency scripts with miniconda as inspiration.
- There should be a build of `gimp` installed into the conda requirements.
- If the community is interested in doing reverse-engineering using imaging tools, strongly recommend to add `Degate` as a package.
- This project uses `Magic`, `Klayout`, `yosys`, `openroad`, `netgen`, and `sky130`. Credit to their respective authors. Their projects are liked in the content of this notebook.